In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import numpy as np
import pickle

from shapely.geometry import Point, mapping
from shapely.geometry import box

import folium
import json



## Methods

In [2]:
def buffer_generation(gdf_POI, uniqueID, BUFFER_METERS):

    # select one node from gdf
    selected_node = gdf_POI[gdf_POI['POI_uniqueID'] == uniqueID].geometry
    lon = selected_node.x
    utm_zone = int((lon + 180) / 6) + 1
    epsg_utm = 32600 + utm_zone  # EPSG code for UTM (northern hemisphere)

    # convert to UTM for accurate buffering
    gdf_POI_utm = gdf_POI.to_crs(epsg=epsg_utm)
    selected_node_utm = gdf_POI_utm[gdf_POI_utm['POI_uniqueID'] == uniqueID].geometry

    buffer = selected_node_utm.buffer(BUFFER_METERS)

    boundary_utm = gpd.GeoSeries(buffer).unary_union
    gdf_boundary_utm = gpd.GeoDataFrame(geometry=[boundary_utm], crs=f"EPSG:{epsg_utm}")
    gdf_boundary = gdf_boundary_utm.to_crs(epsg=4326)
    boundary_geom = gdf_boundary.geometry.iloc[0]

    return boundary_geom, gdf_boundary

### parameters

In [3]:
CITY = 'wien'
uniqueID = 3
BUFFER_METERS = 1200 #meters (600 or 1200)
RESPONDER_TYPE = "police" #hospital, fire_station, police

### load spatial and network data

In [4]:
# gdf POI with ID to identify files
gdf_POI = gpd.read_file(f"../data/{CITY}-locations-uniqueID.geojson", driver='GeoJSON')

# association between ID and POI name
dict_POI = gdf_POI.set_index('POI_uniqueID')['name'].to_dict()

POI_name = dict_POI[uniqueID]
POI_uniqueIDs = list(dict_POI.keys())

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [5]:
gdf_POI

,name,latitude,longitude,POI_uniqueID,geometry
0,Strandbad Gänsehäufel,48.228500,16.426667,0,POINT (16.42667 48.2285)
1,Neustift am Walde,48.254143,16.279135,1,POINT (16.27913 48.25414)
2,Schottenring: Höhe Franz-Josefs-Kai Richtung B...,48.214827,16.371254,2,POINT (16.37125 48.21483)
3,Währing,48.231900,16.289571,3,POINT (16.28957 48.2319)
4,Ernst Happel Stadium,48.207208,16.420985,4,POINT (16.42098 48.20721)
5,Allianz Stadion,48.198026,16.263446,5,POINT (16.26345 48.19803)


In [6]:
responder_names = {'hospital':'hospitals','police':'police stations','fire_station':'fire stations'}
color_map = {'hospital':'Reds','police':'Blues','fire_station':'Oranges'}

In [7]:
# poi
gdf_POI_attacked = gdf_POI[gdf_POI['POI_uniqueID'] == uniqueID]
gdf_POI_attacked.set_crs(epsg=4326, inplace=True)

# load specific impact file
merged_gdf = gpd.read_file(f"../processed/{CITY}/impacts_POI_{RESPONDER_TYPE}/{CITY}_impacts_POI_{uniqueID}_buffer_{BUFFER_METERS}.geojson")
merged_gdf = merged_gdf.reset_index()

# generate buffer boundary
boundary_geom, gdf_boundary = buffer_generation(gdf_POI, uniqueID, BUFFER_METERS)

# filter detached areas
merged_gdf_detached = merged_gdf[merged_gdf['IMPACT'].isna()]
merged_gdf['PERC'] = merged_gdf.apply(lambda row: (row['IMPACT']-1.0)*100, axis = 1)


/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/ms/3n67wbb95xn26d6lg9q5sjw40000gn/T/ipykernel_1651/2942476012.py:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  utm_zone = int((lon + 180) / 6) + 1
/var/folders/ms/3n67wbb95xn26d6lg9q5sjw40000gn/T/ipykernel_1651/2942476012.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  boundary_utm = gpd.GeoSeries(buffer).unary_union


### plot folium map

In [8]:

# get POI center
poi_point = gdf_POI_attacked.geometry.iloc[0]
center_lat, center_lon = poi_point.y, poi_point.x

# convert merged_gdf to GeoJSON format
merged_gdf_json = json.loads(merged_gdf.to_json())

m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='openstreetmap')

# add buffer boundary
for _, boundary in gdf_boundary.iterrows():
    if boundary.geometry.geom_type == "Polygon":
        coords = list(boundary.geometry.exterior.coords)
        folium.Polygon(locations=[(lat, lon) for lon, lat in coords], color="red", weight=3, fill=False).add_to(m)

# poi
folium.Marker(
    location=[center_lat, center_lon],
    popup=f"POI: {POI_name}",
    icon=folium.Icon(color="red")
).add_to(m)

# delay layer
folium.Choropleth(
    geo_data=merged_gdf_json,
    name="Impact Zone",
    data=merged_gdf,
    columns=["index", "PERC"],
    key_on="feature.id",
    fill_color=color_map[RESPONDER_TYPE],
    fill_opacity=0.75,
    line_opacity=0.1,
    #legend_name=f"Delay Index in Accessibility to {responder_names[RESPONDER_TYPE]}",
    legend_name=f"[%] Increase in travel time to {responder_names[RESPONDER_TYPE]}",
    highlight = True
).add_to(m)

# add Detached Areas if they exist
if len(merged_gdf_detached) > 0:
    merged_gdf_detached = merged_gdf_detached.reset_index()
    merged_gdf_detached_json = json.loads(merged_gdf_detached.to_json())

    folium.GeoJson(
        merged_gdf_detached_json,
        name="Detached Areas",
        style_function=lambda feature: {
            "fillColor": "gray",
            "color": "gray",
            "weight": 1,
            "fillOpacity": 0.25,
        },
        tooltip=folium.GeoJsonTooltip(fields=["index"], aliases=["Detached Area Index"])
    ).add_to(m)


info_box_html = f'''
 <div style="position: fixed; bottom: 150px; left: 50px; width: 250px; background-color: white; z-index:9999; padding: 12px; 
             font-size:14px; border-radius: 5px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3); text-align: justify;">
     <b>Urban Resiliency</b><br>
     Accessibility delays to emergency responders after roads around a Point of Interest are blocked. 
     Areas highlighted indicate increased travel times to <b>{responder_names[RESPONDER_TYPE]}</b> due to the road network disruption.<br>
     - POI selected: <b>{POI_name}</b><br>
 </div>
'''

legend_html = f'''
 <div style="position: fixed; bottom: 50px; left: 50px; width: 220px; background-color: white; z-index:9999; padding: 10px; font-size:14px; border-radius: 5px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);">
     <b>Legend</b><br>
     <i style="background: red; width: 12px; height: 2px; display: inline-block; margin-right: 8px;"></i> No-Access Area (Radius of <b>{BUFFER_METERS} m</b>)<br>
'''

if len(merged_gdf_detached) > 0:
    legend_html += '''<i style="background: gray; width: 12px; height: 12px; float: left; margin-right: 8px;"></i> Detached zone after disruption<br>'''

legend_html += '</div>'

m.get_root().html.add_child(folium.Element(info_box_html))

m.get_root().html.add_child(folium.Element(legend_html))

m.get_root().html.add_child(folium.Element("""
    <style>
        .leaflet-control.legend {
            font-size: 16px !important;  /* Adjust size */
            font-weight: bold;           /* Optional: Make it bold */
        }
    </style>
"""))

m

#m.save(f"../figures/maps/{CITY}/POI_{uniqueID}/{CITY}_POI_{uniqueID}_buffer_{BUFFER_METERS}_{RESPONDER_TYPE}.html")


